## Define helper functions and load modules

In [1]:
import keras
import tensorflow as tf
import Bio.SeqIO as SeqIO
import random
import numpy as np
import sys
import pandas as pd
import tqdm
from keras.models import Sequential 
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout, Bidirectional, BatchNormalization
from keras.callbacks import EarlyStopping
import wandb
from utils.CNN_utils import remove_N, onehote

def balanced_accuracy(y_true, y_pred):
    # Convert tensors to NumPy arrays for processing
    y_true = tf.make_ndarray(y_true)
    y_pred = tf.make_ndarray(y_pred)

    # Calculate confusion matrix
    confusion = tf.math.confusion_matrix(y_true, y_pred, num_classes=2)

    # Calculate sensitivity (true positive rate) for each class
    tp = confusion[1, 1]
    fn = confusion[1, 0]
    sensitivity = tp / (tp + fn)

    # Calculate the balanced accuracy as the average sensitivity
    balanced_acc = sensitivity

    return balanced_acc

2024-01-16 10:47:04.485071: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 10:47:04.485158: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 10:47:04.485205: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 10:47:04.495570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Run code for short sequence training

In [2]:
MAX_LEN=3000
MIN_LEN=0

LTRs = [rec for rec in SeqIO.parse("train_LTRs.fasta", "fasta") if len(rec.seq) < MAX_LEN and len(rec.seq) > MIN_LEN]
n_sequences = len(LTRs)

non_LTRs = [rec for rec in SeqIO.parse("train_nonLTRs.fasta", "fasta") if len(rec.seq) < MAX_LEN and len(rec.seq) > MIN_LEN]

sequences = [onehote(remove_N(str(rec.seq))) for rec in tqdm.tqdm(non_LTRs+LTRs)]
labels = [0]*len(non_LTRs) + [1]*len(LTRs)

100%|██████████| 283064/283064 [02:13<00:00, 2121.83it/s]


In [3]:
from sklearn.model_selection import train_test_split

# Split into train and test
paddedDNA = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="pre", maxlen=MAX_LEN)
trainX, valX, trainY, valY = train_test_split(paddedDNA, labels, test_size=0.1, random_state=42)


In [24]:
model2 = Sequential()

model2.add(Conv1D(filters=32, kernel_size=16, padding='same', activation='relu', input_shape=trainX[0].shape))
model2.add(Dropout(0.2))  # You can adjust the dropout rate as needed
model2.add(MaxPooling1D(pool_size=4))
model2.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model2.add(Dropout(0.2))  # You can adjust the dropout rate as needed
model2.add(MaxPooling1D(pool_size=4))
model2.add(Flatten())
model2.add(Dense(units=256, activation='relu'))
model2.add(Dense(units=1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'], weighted_metrics=["binary_accuracy"])

#model2.fit(valX, np.array(valY), epochs=3, batch_size=64,verbose = 1,validation_data=(valX, np.array(valY)), callbacks=[WandbCallback()])
model2.fit(trainX, np.array(trainY).reshape(-1, 1), epochs=15, batch_size=64,verbose = 1,validation_data=(valX, np.array(valY).reshape(-1, 1)), callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/15
3981/3981 [==============================] - 26s 6ms/step - loss: 0.5101 - binary_accuracy: 0.7404 - weighted_binary_accuracy: 0.7404 - val_loss: 0.4512 - val_binary_accuracy: 0.7772 - val_weighted_binary_accuracy: 0.7772
Epoch 2/15
3981/3981 [==============================] - 23s 6ms/step - loss: 0.4195 - binary_accuracy: 0.7998 - weighted_binary_accuracy: 0.7998 - val_loss: 0.4097 - val_binary_accuracy: 0.8057 - val_weighted_binary_accuracy: 0.8057
Epoch 3/15
3981/3981 [==============================] - 23s 6ms/step - loss: 0.3878 - binary_accuracy: 0.8168 - weighted_binary_accuracy: 0.8168 - val_loss: 0.3978 - val_binary_accuracy: 0.8191 - val_weighted_binary_accuracy: 0.8191
Epoch 4/15
3981/3981 [==============================] - 32s 8ms/step - loss: 0.3645 - binary_accuracy: 0.8303 - weighted_binary_accuracy: 0.8303 - val_loss: 0.3793 - val_binary_accuracy: 0.8225 - val_weighted_binary_accuracy: 0.8225
Epoch 5/15
3981/3981 [==============================] - 23s 6ms/step

In [23]:
model2.save("all_length_cnn_lstm_for_SHAP.h5")

/var/tmp/xhorvat9/tf_CUDA/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
